# Construcción de la base de datos para el modelo de regresión lineal

En este notebook podrás encontrar el procedimiento realizado para construir la base de datos que utilizaremos en el tutorial de **Regresión Lineal**. Los datos que utilizaremos son provenientes del Banco Mundial.

**Nota:** Para descargar los datos directamente desde el banco mundial puedes ir al siguiente enlace: https://datos.bancomundial.org/indicador

### Importar librerias

In [26]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Configurar warnings
import warnings
warnings.filterwarnings('ignore')

### Seleccionar sólo los países de las bases de datos del banco mundial

En las bases de datos del banco mundial se muestran los datos de países, continentes, regiones e incluso en mundo, es por esto  que debemos aplicar un filtrado de datos para utilizar unicamente los países.

In [27]:
paises_iso = [
    "AFG", "ALB", "DZA", "AND", "AGO", "ATG", "ARG", "ARM", "AUS", "AUT",
    "AZE", "BHS", "BHR", "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BTN",
    "BOL", "BIH", "BWA", "BRA", "BRN", "BGR", "BFA", "BDI", "CPV", "KHM",
    "CMR", "CAN", "CAF", "TCD", "CHL", "CHN", "COL", "COM", "COD", "COG",
    "CRI", "HRV", "CUB", "CYP", "CZE", "DNK", "DJI", "DMA", "DOM", "ECU",
    "EGY", "SLV", "GNQ", "ERI", "EST", "SWZ", "ETH", "FJI", "FIN", "FRA",
    "GAB", "GMB", "GEO", "DEU", "GHA", "GRC", "GRD", "GTM", "GIN", "GNB",
    "GUY", "HTI", "HND", "HUN", "ISL", "IND", "IDN", "IRN", "IRQ", "IRL",
    "ISR", "ITA", "JAM", "JPN", "JOR", "KAZ", "KEN", "KIR", "PRK", "KOR",
    "KWT", "KGZ", "LAO", "LVA", "LBN", "LSO", "LBR", "LBY", "LIE", "LTU",
    "LUX", "MDG", "MWI", "MYS", "MDV", "MLI", "MLT", "MHL", "MRT", "MUS",
    "MEX", "FSM", "MDA", "MCO", "MNG", "MNE", "MAR", "MOZ", "MMR", "NAM",
    "NRU", "NPL", "NLD", "NZL", "NIC", "NER", "NGA", "MKD", "NOR", "OMN",
    "PAK", "PLW", "PAN", "PNG", "PRY", "PER", "PHL", "POL", "PRT", "QAT",
    "ROU", "RUS", "RWA", "KNA", "LCA", "VCT", "WSM", "SMR", "STP", "SAU",
    "SEN", "SRB", "SYC", "SLE", "SGP", "SVK", "SVN", "SLB", "SOM", "ZAF",
    "SSD", "ESP", "LKA", "SDN", "SUR", "SWE", "CHE", "SYR", "TJK", "THA",
    "TLS", "TGO", "TON", "TTO", "TUN", "TUR", "TKM", "TUV", "UGA", "UKR",
    "ARE", "GBR", "USA", "URY", "UZB", "VUT", "VAT", "VEN", "VNM", "YEM",
    "ZMB", "ZWE"
]

### Seleccionar las variables a utilizar

In [28]:
v_independientes = ['crecimiento_pob', 'pib_per_capita', 'impuestos']
v_dependiente = "gasto_publico"

year = "2019"

### Importar variables a utilizar

En este punto leemos las bases de datos y seleccionamos las  variables que necesitamos, como el código ISO, el nombre del país y la variable de interés

In [29]:
# PIB PER CAPITA
pib_per_capita = pd.read_excel("./datos/pib_per_capita.xls", sheet_name="Data", skiprows=3)
pib_per_capita = pib_per_capita[pib_per_capita["Country Code"].isin(paises_iso)]
pib_per_capita = pib_per_capita[["Country Code", "Country Name", year]]
pib_per_capita.columns = ["codigo_pais", "nombre_pais", "pib_per_capita"]

# CRECIMIENTO POBLACIONAL
crecimiento_pob = pd.read_excel("./datos/crecimiento_poblacional.xls", sheet_name="Data", skiprows=3)
crecimiento_pob = crecimiento_pob[crecimiento_pob["Country Code"].isin(paises_iso)]
crecimiento_pob = crecimiento_pob[["Country Code", "Country Name", year]]
crecimiento_pob.columns = ["codigo_pais", "nombre_pais", "crecimiento_pob"]

# GASTO PUBLICO
gasto_publico = pd.read_excel("./datos/gasto_publico.xls", sheet_name="Data", skiprows=3)
gasto_publico = gasto_publico[gasto_publico["Country Code"].isin(paises_iso)]
gasto_publico = gasto_publico[["Country Code", "Country Name", year]]
gasto_publico.columns = ["codigo_pais", "nombre_pais", "gasto_publico"]

# IMPUESTOS
impuestos = pd.read_excel("./datos/impuestos.xls", sheet_name="Data", skiprows=3)
impuestos = impuestos[impuestos["Country Code"].isin(paises_iso)]
impuestos = impuestos[["Country Code", "Country Name", year]]
impuestos.columns = ["codigo_pais", "nombre_pais", "impuestos"]

# PEGADO DE DATOS
df_regresion = pd.merge(crecimiento_pob, pib_per_capita, on=["codigo_pais", "nombre_pais"], how="inner")
df_regresion = pd.merge(df_regresion, gasto_publico, on=["codigo_pais", "nombre_pais"], how="inner")
df_regresion = pd.merge(df_regresion, impuestos, on=["codigo_pais", "nombre_pais"], how="inner")

df_regresion = df_regresion.dropna()

print(df_regresion.isna().sum())
print(len(df_regresion))


codigo_pais        0
nombre_pais        0
crecimiento_pob    0
pib_per_capita     0
gasto_publico      0
impuestos          0
dtype: int64
135


### Método IQR para eliminar outliers

In [30]:
def eliminar_outliers(df, columnas):
  for columna in columnas:
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    filtro = (df[columna] >= (Q1 - 1.5 * IQR)) & (df[columna] <= (Q3 + 1.5 * IQR))
    df = df[filtro]
  return df

columnas_a_filtrar = v_independientes + [v_dependiente]
df_regresion = eliminar_outliers(df_regresion, columnas_a_filtrar)
len(df_regresion)

122

### Creamos una nueva base de datos con las modificaciones realizadas

In [31]:
df_regresion.to_csv("./datos/df_regresion.csv", index=False)